In [10]:
import requests
import pandas as pd
# from bs4 import BeautifulSoup
# import pandas as pd
# import numpy as np
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

import altair as alt
import lxml

In [41]:
records = []

for period in range(1, 39):  # periods 1–38
    payload = {
        "msgs": [
            {
                "method": "getStandings",
                "data": {
                    "leagueId": "962c1utjlzbm8vb8",
                    "view": "REGULAR_SEASON",
                    "timeframeType": "BY_PERIOD",
                    "timeStartType": "FROM_SEASON_START",
                    "period": str(period)
                }
            }
        ],
        "uiv": 3,
        "refUrl": f"https://www.fantrax.com/fantasy/league/962c1utjlzbm8vb8/standings;view=REGULAR_SEASON;timeframeType=BY_PERIOD;timeStartType=FROM_SEASON_START;period={period}",
        "dt": 1,
        "at": 0,
        "av": "0.0",
        "tz": "America/Los_Angeles",
        "v": "167.0.1"
    }

    r = requests.post(url, headers=headers, json=payload)
    r.raise_for_status()
    j = r.json()

    data = j["responses"][0]["data"]
    team_info = data.get("fantasyTeamInfo", {})

    # find the "Standings" table
    standings_tbl = next(
        (t for t in data.get("tableList", []) if t.get("caption") == "Standings"),
        None
    )
    if not standings_tbl:
        print(f"Period {period}: Standings table not found")
        continue

    for row in standings_tbl.get("rows", []):
        fixed = row.get("fixedCells", [])
        cells = row.get("cells", [])

        # fixed cells: [rank, team cell]
        rank = fixed[0].get("content") if len(fixed) > 0 else None
        team_cell = fixed[1] if len(fixed) > 1 else {}
        team_name = team_cell.get("content")
        team_id = team_cell.get("teamId")

        ti = team_info.get(team_id, {})
        record = {
            "gw": period,
            "rank": rank,
            "teamId": team_id,
            "team": team_name,
            "shortName": ti.get("shortName"),
            "logo": ti.get("logoUrl512"),
            # header order for cells: W, D, L, Points, Win%, WW, FPtsF, FPtsA, Streak
            "W": cells[0].get("content") if len(cells) > 0 else None,
            "D": cells[1].get("content") if len(cells) > 1 else None,
            "L": cells[2].get("content") if len(cells) > 2 else None,
            "Points": cells[3].get("content") if len(cells) > 3 else None,
            "Win%": cells[4].get("content") if len(cells) > 4 else None,
            "WW": cells[5].get("content") if len(cells) > 5 else None,
            "FPtsF": cells[6].get("content") if len(cells) > 6 else None,
            "FPtsA": cells[7].get("content") if len(cells) > 7 else None,
            "Streak": cells[8].get("content") if len(cells) > 8 else None,
        }
        records.append(record)

df = pd.DataFrame(records)

# make numeric columns numeric
for col in ["rank", "W", "D", "L", "Points", "Win%", "WW", "FPtsF", "FPtsA"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [43]:
df_trim = df[['W', 'D', 'L','Points','Win%', 'WW', 'FPtsF', 'FPtsA', 'Streak','team','rank','gw']]

In [44]:
data = df_trim

chart = alt.Chart(data).mark_line(point=True).encode(
    x = alt.X('gw', scale=alt.Scale(domain=[0, 38]), title="Gameweek"),
    y=alt.Y('rank', aggregate={'argmax': 'gw'}, scale=alt.Scale(domain=[10,1]), title="Rank"),
    color=alt.Color("team", legend=None),
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="ascending")],
    groupby=["gw"]
).properties(
    title="Fake Internet Soccer XIV standings by gameweek",
    width=700,
    height=350,
)

labels = alt.Chart(data).mark_text(
    align='left', dx=5
).encode(
    x = alt.X('max(gw)', scale=alt.Scale(domain=[0, 38])),
    y=alt.Y('rank', aggregate={'argmax': 'gw'}, scale=alt.Scale(domain=[10,1])),
    text='team:N',
    color='team:N',
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="ascending")],
    groupby=["gw"]
)

chart + labels

alt.LayerChart(...)

In [46]:
chart = alt.Chart(df_trim).mark_line(point=True).encode(
    x = alt.X('gw', scale=alt.Scale(domain=[0, 38]), title="Gameweek"),
    y=alt.Y("Points", title="Points"),
    color=alt.Color("team", legend=None),
    tooltip=["team"]
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="descending")],
    groupby=["gw"]
).properties(
    title="Fake Internet Soccer XIV standings by gameweek",
    width=700,
    height=350,
)

labels = alt.Chart(df_trim).mark_text(
    align='left', dx=5
).encode(
    x = alt.X('max(gw)', scale=alt.Scale(domain=[0, 38])),
    y=alt.Y('Points', aggregate={'argmax': 'gw'}),
    text='team:N',
    color='team:N',
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="descending")],
    groupby=["gw"]
)

chart + labels

alt.LayerChart(...)

In [50]:
df_trim.to_csv("data/output/standings/standings-season-14.csv", index=False)

To JSON

In [51]:
clean_df = df_trim.reset_index()[["team","rank","gw","Points"]]

In [52]:
clean_df.columns = clean_df.columns.str.lower()

In [53]:
clean_df.to_json("../_data/standings-xiv.json", orient='records')